In [4]:
!pip install -U langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.9 MB/s eta 0:00:00


In [2]:
!pip install langchain gradio faiss-cpu sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 439.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.7/56.7 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [122]:
from getpass import getpass
import gradio as gr
import os
import requests

def get_api_token():
    """Prompts the user for a Hugging Face Hub API token."""
    print("**Please enter your Hugging Face Hub API token:**")
    token = getpass("Token: ")
    return token

# Set your Hugging Face Hub API token here
os.environ["HUGGINGFACEHUB_API_TOKEN"] = get_api_token()  # Secure token input

def query_model(document, question):
    # Prepare the input for the model
    prompt = f"""You are a helpful legal assistant. Consider this context:

Document: {document}
Question: {question}

Provide a clear, accurate, and user-friendly response that:
- Explains relevant legal concepts and principles
- Provides practical advice and recommendations
- Uses simple language and examples
- Maintains professional integrity

only answer the response and say nothing else. kukuduku
"""

    headers = {
        "Authorization": f"Bearer {os.environ['HUGGINGFACEHUB_API_TOKEN']}",
        "Content-Type": "application/json"
    }

    # Send request to the Hugging Face model
    response = requests.post(
        "https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3-8B-Instruct",
        headers=headers,
        json={"inputs": prompt}
    )

    if response.status_code == 200:
        # Extract only the generated text from the response
        generated_text = response.json()[0]['generated_text']

        # Find where the actual answer starts (after "kukuduku")
        start_index = generated_text.find("kukuduku")

        if start_index != -1:
            # Return only what comes after this phrase
            return generated_text[start_index + len("kukuduku"):].strip()

        return generated_text.strip()  # Return full text if no specific phrase found
    else:
        return f"Error: {response.status_code}, {response.text}"

def create_gradio_interface():
    """Creates the Gradio interface."""
    with gr.Blocks(title="Legal Query Assistant") as demo:
        gr.Markdown("# Legal Query Assistant")

        with gr.Row():
            document_input = gr.Textbox(
                label="Legal Document",
                placeholder="Paste your legal document here...",
                lines=10
            )
            question_input = gr.Textbox(
                label="Your Question",
                placeholder="Ask a question about the document...",
                lines=2
            )

        with gr.Row():
            submit_btn = gr.Button("Submit Question", variant="primary")

        with gr.Row():
            output_box = gr.Textbox(
                label="Response",
                lines=8,
                interactive=False
            )

        submit_btn.click(lambda doc, ques: query_model(doc, ques), inputs=[document_input, question_input], outputs=output_box)

    return demo

if __name__ == "__main__":
    demo = create_gradio_interface()
    demo.launch(share=True, debug=True)

**Please enter your Hugging Face Hub API token:**
Token: ··········
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://0d66cfc85d5c4a1f7f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://0d66cfc85d5c4a1f7f.gradio.live
